# Machine Learning Project on Github Issues
+ Github Issue Classification
+ Predicting if Github Issue is a Bug,Enhancement,Question or Docs,etc.
+ Data Source
+ https://tickettagger.blob.core.windows.net/datasets/dataset-labels-top3-30k-real.txt
+ https://gist.githubusercontent.com/rafaelkallis/6aa281b00d73d77fc843bd34f8184854/raw/8c10ebf2fd6f937f8
+ https://github.com/dotnet/samples/tree/main/machine-learning/tutorials/GitHubIssueClassification/Data
+ Credits & Derivative Work
+ httpsBEE, Yang Song and Oscar Chaparro
+ Tim Moser, David Steiger, Christian Birchler, Lara Fried, Sebastiano Panichella, Rafael Kallis